In [1]:
import pandas as pd
from datetime import datetime
import sql_functions as sql_int

In [2]:
def new_tournament(tournament_add,name,short_name,start,fin):
    start = datetime.strptime(start, '%Y-%m-%d')
    fin = datetime.strptime(fin, '%Y-%m-%d')
    tournament_add.append([name,short_name,start,fin])
def add_tournament(tournament_lst,tournament_df):
    df_add = pd.DataFrame(tournament_lst,columns=['name','short_name','start','fin'])
    df_add.drop_duplicates(inplace=True)
    tournament_df = pd.concat([tournament_df,df_add],ignore_index=True)
    return tournament_df

## New tournament
#### Read new tournaments from CSV-file

In [9]:
tournament_add = []
tournaments = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/tournaments.csv',
                            dtype={'name' : 'str', 'short_name' : 'str' },
                            parse_dates=["start", "fin"])
df_tournaments = tournaments.copy()
df_tournaments[['name','short_name']] = df_tournaments[['name','short_name']].apply(lambda x : x.str.lower())

#### Example for new tournament

In [4]:
new_tournament(tournament_add,
               'Rising Legends Runeterra Reforged - Open Qualifier 2 Day 2',
               'RL RR OQ2 D2',
               '2023-08-06',
               '2023-08-06'
               )

In [5]:
df_tournaments = add_tournament(tournament_add,df_tournaments)
df_tournaments[['name','short_name']] = df_tournaments[['name','short_name']].apply(lambda x : x.str.lower())

## Get existing Tournaments

In [4]:
schema = 'tft_tables'
table = 'tournaments'
sql_query = f'select * from {schema}.{table};'
existing_tournaments = pd.DataFrame(sql_int.get_data_mysql(sql_query),
                                  columns=['tournament_id','name','short_name','start','fin'])

df_existing_tournaments = existing_tournaments.copy()
df_existing_tournaments[['start','fin']] = df_existing_tournaments[['start','fin']].apply(pd.to_datetime)
df_existing_tournaments['existing'] = 'Yes'
df_existing_tournaments[['name','short_name']] = df_existing_tournaments[['name','short_name']].apply(lambda x : x.str.lower())
df_tournaments = pd.concat([df_existing_tournaments,df_tournaments],ignore_index=True)

In [5]:
df_tournaments[df_tournaments.duplicated(keep = False, subset=['name']) == True].sort_values(by=['name','short_name'])

,tournament_id,name,short_name,start,fin,existing
5,6.0,rising legends runeterra reforged - golden spa...,rl rr gs1 d1,2023-07-14,2023-07-14,Yes
14,NaN,rising legends runeterra reforged - golden spa...,rl rr gs1 d1,2023-07-14,2023-07-14,NaN
6,7.0,rising legends runeterra reforged - golden spa...,rl rr gs1 d2,2023-07-15,2023-07-15,Yes
15,NaN,rising legends runeterra reforged - golden spa...,rl rr gs1 d2,2023-07-15,2023-07-15,NaN
7,8.0,rising legends runeterra reforged - golden spa...,rl rr gs1 d3,2023-07-16,2023-07-16,Yes
16,NaN,rising legends runeterra reforged - golden spa...,rl rr gs1 d3,2023-07-16,2023-07-16,NaN
1,2.0,rising legends runeterra reforged - open quali...,rl rr oq1 d1 ga,2023-07-08,2023-07-08,Yes
10,NaN,rising legends runeterra reforged - open quali...,rl rr oq1 d1 ga,2023-07-08,2023-07-08,NaN
2,3.0,rising legends runeterra reforged - open quali...,rl rr oq1 d1 gb,2023-07-08,2023-07-08,Yes
11,NaN,rising legends runeterra reforged - open quali...,rl rr oq1 d1 gb,2023-07-08,2023-07-08,NaN


In [6]:
df_tournaments = (df_tournaments
                  .drop_duplicates(subset=['name'],keep=False)
                  .query('existing != "Yes"')
                  .drop(['tournament_id','existing'],axis=1)
                  )

push new tournaments into Database

In [8]:
sql_int.push_to_database(df_tournaments,table,sql_int.get_engine_alchemy(),schema)

The tournaments table was imported successfully.
